# How to simply use keras
* Reference
    + https://www.tensorflow.org/guide/keras

## Setup

In [1]:
import numpy as np
import tensorflow as tf

keras = tf.keras

print(tf.__version__)
print(tf.keras.__version__)

1.12.0
2.1.6-tf


## Build a simple model
### Sequential model
In Keras, you assemble layers to build models. A model is (usually) a graph of layers. The most common type of model is a stack of layers: the `tf.keras.Sequential` model.

In [2]:
## To build a simple, fully-connected network (i.e. multi-layer perceptron)
# If you specify the input shape, the model gets built continuously, as you are adding layers.
# Note that when using this delayed-build pattern (no input shape specified),
# the model doesn't have any weights until the first call,
# to a training/evaluation method (since it isn't yet built)

model = keras.Sequential()
model.add(keras.layers.Dense(units = 64, activation = 'relu')) 
# or model.add(keras.layers.Dense(units = 64, activation = 'relu'))
model.add(keras.layers.Dense(units = 64, activation = 'relu'))
model.add(keras.layers.Dense(units = 10, activation = 'softmax'))

In [3]:
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[]


### Configure the layers
There are many tf.keras.layers available with some common constructor parameters:

* `activation`: Set the activation function for the layer. This parameter is specified by the name of a built-in function or as a callable object. By default, no activation is applied.
* `kernel_initializer` and `bias_initializer`: The initialization schemes that create the layer's weights (kernel and bias). This parameter is a name or a callable object. This defaults to the `"Glorot uniform"` initializer.
* `kernel_regularizer` and `bias_regularizer`: The regularization schemes that apply the layer's weights (kernel and bias), such as L1 or L2 regularization. By default, no regularization is applied.  
  
The following instantiates `tf.keras.layers.Dense` layers using constructor arguments:

In [4]:
keras.backend.clear_session()
tf.reset_default_graph()
# Create a sigmoid layer:
keras.layers.Dense(64, activation='sigmoid')
# Or:
keras.layers.Dense(64, activation=tf.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
keras.layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
keras.layers.Dense(64, kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
keras.layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

In [5]:
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[]


## Train and evaluate

### Set up training
After the model is constructed, configure its learning process by calling the `compile` method, `tf.keras.Model.compile` or `tf.keras.Sequential.compile` takes three important arguments.  
  
* `optimizer`: This object specifies the training procedure. Pass it optimizer instances from the `tf.train` module, such as `tf.train.AdamOptimizer`, `tf.train.RMSPropOptimizer`, or `tf.train.GradientDescentOptimizer`.
* `loss`: The function to minimize during optimization. Common choices include mean square error (`mse`), `categorical_crossentropy`, and `binary_crossentropy`. Loss functions are specified by name or by passing a callable object from the `tf.keras.losses` module.
* `metrics`: Used to monitor training. These are string names or callables from the `tf.keras.metrics` module.

In [6]:
keras.backend.clear_session()
tf.reset_default_graph()

model = keras.Sequential()
model.add(keras.layers.Dense(units=64, activation='relu', input_shape = (32,)))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=10, activation='softmax'))

print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[<tf.Variable 'dense/kernel:0' shape=(32, 64) dtype=float32>, <tf.Variable 'dense/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'dense_1/kernel:0' shape=(64, 64) dtype=float32>, <tf.Variable 'dense_1/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'dense_2/kernel:0' shape=(64, 10) dtype=float32>, <tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32>]


In [7]:
# Compile
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[<tf.Variable 'dense/kernel:0' shape=(32, 64) dtype=float32>, <tf.Variable 'dense/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'dense_1/kernel:0' shape=(64, 64) dtype=float32>, <tf.Variable 'dense_1/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'dense_2/kernel:0' shape=(64, 10) dtype=float32>, <tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32>, <tf.Variable 'TFOptimizer/iterations:0' shape=() dtype=int64>]


The following shows a few examples of configuring a model for training:

In [8]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# Configure a model for categorical classification.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

### Input NumPy data

In [9]:
keras.backend.clear_session()
tf.reset_default_graph()

# Numpy dataset
tr_data = np.random.random((1000, 32))
tr_label = np.random.randint(low=0, high=10, size = 1000)

val_data = np.random.random((100, 32))
val_label = np.random.randint(low=0, high=10, size = 100)

tst_data = np.random.random((100, 32))
tst_label = np.random.randint(low=0, high=10, size = 100)

print(tr_data.dtype, tr_label.dtype)

float64 int64


In [10]:
# Create a model
model = keras.Sequential()
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=10, activation='softmax'))
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

[]


In [11]:
model.compile(optimizer=tf.train.GradientDescentOptimizer(.01), 
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

model.fit(x=tr_data, y=tr_label, epochs=5, batch_size=32, validation_data=(val_data, val_label))

[<tf.Variable 'TFOptimizer/iterations:0' shape=() dtype=int64>]
Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 1s 644us/step - loss: 2.3531 - acc: 0.1030 - val_loss: 2.3664 - val_acc: 0.0700
Epoch 2/5
1000/1000 [==============================] - 0s 40us/step - loss: 2.3258 - acc: 0.1020 - val_loss: 2.3476 - val_acc: 0.0900
Epoch 3/5
1000/1000 [==============================] - 0s 41us/step - loss: 2.3128 - acc: 0.0950 - val_loss: 2.3410 - val_acc: 0.0700
Epoch 4/5
1000/1000 [==============================] - 0s 40us/step - loss: 2.3069 - acc: 0.1030 - val_loss: 2.3368 - val_acc: 0.0700
Epoch 5/5
1000/1000 [==============================] - 0s 49us/step - loss: 2.3027 - acc: 0.1040 - val_loss: 2.3348 - val_acc: 0.0700


In [12]:
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

[<tf.Variable 'TFOptimizer/iterations:0' shape=() dtype=int64>, <tf.Variable 'dense/kernel:0' shape=(32, 64) dtype=float32>, <tf.Variable 'dense/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'dense_1/kernel:0' shape=(64, 64) dtype=float32>, <tf.Variable 'dense_1/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'dense_2/kernel:0' shape=(64, 10) dtype=float32>, <tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32>]


In [13]:
# Evaluate and predict
print(model.metrics_names)
print(model.evaluate(x=tst_data, y=tst_label))
print(model.predict(x=tst_data).shape)

['loss', 'acc']
100/100 [==============================] - 0s 56us/step
[2.3359627723693848, 0.06]
(100, 10)


### Input tf.data datasets
Pass a `tf.data.Dataset` instance to the `fit`, `evaluate`, `predict` method.

In [14]:
keras.backend.clear_session() # very important!
tf.reset_default_graph()

print(tf.get_default_graph().get_operations())
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[]
[]


In [15]:
# tf.data.Dataset instance
tr_data = np.random.random((1000, 32)).astype(np.float32)
tr_label = np.random.randint(low=0, high=10, size = 1000)
tr_dataset = tf.data.Dataset.from_tensor_slices((tr_data, tr_label))
tr_dataset = tr_dataset.batch(batch_size=32)
tr_dataset = tr_dataset.repeat()

val_data = np.random.random((100, 32)).astype(np.float32)
val_label = np.random.randint(low=0, high=10, size = 100)
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
val_dataset = val_dataset.batch(batch_size=100).repeat()

tst_data = np.random.random((100, 32)).astype(np.float32)
tst_label = np.random.randint(low=0, high=10, size = 100)
tst_dataset = tf.data.Dataset.from_tensor_slices((tst_data, tst_label))
tst_dataset = tst_dataset.batch(batch_size=100)

print(tr_dataset.output_types)

(tf.float32, tf.int64)


In [16]:
# Training
model = keras.Sequential()
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=10, activation='softmax'))
model.compile(optimizer=tf.train.GradientDescentOptimizer(.01), 
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

model.fit(tr_dataset, epochs = 5, steps_per_epoch = 1000 // 32,
          validation_data = val_dataset, validation_steps = 1)

Epoch 1/5
31/31 [==============================] - 0s 3ms/step - loss: 2.3285 - acc: 0.1865 - val_loss: 2.3348 - val_acc: 0.0000e+00
Epoch 2/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3120 - acc: 0.1794 - val_loss: 2.3027 - val_acc: 0.0000e+00
Epoch 3/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3049 - acc: 0.1724 - val_loss: 2.2856 - val_acc: 0.0000e+00
Epoch 4/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3009 - acc: 0.1583 - val_loss: 2.2753 - val_acc: 0.0000e+00
Epoch 5/5
31/31 [==============================] - 0s 1ms/step - loss: 2.2982 - acc: 0.1492 - val_loss: 2.2705 - val_acc: 0.0000e+00


In [17]:
# Evaluate and predict
print(model.metrics_names)
print(model.evaluate(tst_dataset, steps = 1))
print(model.predict(tst_dataset, steps = 1).shape)

['loss', 'acc']
1/1 [==============================] - 0s 887us/step
[2.302539587020874, 0.21875]
(32, 10)


## Build advanced models

### Functional API
The `tf.keras.Sequential` model is a simple stack of layers that cannot represent arbitrary models. Use the Keras functional API to build complex model topologies such as:

* Multi-input models,
* Multi-output models,
* Models with shared layers (the same layer called several times),
* Models with non-sequential data flows (e.g. residual connections).  

Building a model with the functional API works like this:  

1. A layer instance is callable and returns a tensor.
2. Input tensors and output tensors are used to define a `tf.keras.Model` instance.
3. This model is trained just like the `Sequential` model.  

The following example uses the functional API to build a simple, fully-connected network:

In [18]:
# Clear
keras.backend.clear_session()
tf.reset_default_graph()

In [19]:
data = np.random.random((1000, 32)).astype(np.float32)
label = np.random.randint(low=0, high=10, size = 1000).astype(np.int32)
dataset = tf.data.Dataset.from_tensor_slices((data, label))
dataset = dataset.batch(batch_size=32).repeat()
print(dataset.output_types)

inputs = tf.keras.Input(shape=(32,))  # Returns a placeholder tensor
print(inputs, type(inputs))

# A layer instance is callable on a tensor, and returns a tensor.
x = keras.layers.Dense(64, activation='relu')(inputs)
x = keras.layers.Dense(64, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

# Instantiate the model given inputs and outputs
model = keras.Model(inputs = inputs, outputs = predictions)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.train.RMSPropOptimizer(.001),
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# Trains for 5 epochs
model.fit(dataset, epochs=5, steps_per_epoch = 1000//32)

(tf.float32, tf.int32)
Tensor("input_1:0", shape=(?, 32), dtype=float32) <class 'tensorflow.python.framework.ops.Tensor'>
Epoch 1/5
31/31 [==============================] - 0s 4ms/step - loss: 2.3773 - acc: 0.0958
Epoch 2/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3450 - acc: 0.0917
Epoch 3/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3238 - acc: 0.0917
Epoch 4/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3140 - acc: 0.1028
Epoch 5/5
31/31 [==============================] - 0s 2ms/step - loss: 2.3030 - acc: 0.1069


### Model subclassing
Build a fully-customizable model by subclassing `tf.keras.Model` and defining your own forward pass. Create layers in the `__init__` method and set them as attributes of the class instance. Define the forward pass in the call method.

Model subclassing is particularly useful when eager execution is enabled since the forward pass can be written imperatively.

In [20]:
# Clear
keras.backend.clear_session()
tf.reset_default_graph()

print(tf.get_default_graph().get_operations())
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[]
[]


In [21]:
# Subclassing tf.keras.Model
class MLP(keras.Model):
    def __init__(self, hidden_dim, num_classes):
        super(MLP, self).__init__()
        # Define your layers here.
        self.hidden_layer = keras.layers.Dense(units = hidden_dim, activation='relu')
        self.output_layer = keras.layers.Dense(units = num_classes, activation='softmax')
        
    def call(self, inputs):
        hidden = self.hidden_layer(inputs)
        score = self.output_layer(hidden)
        return score
    
# Instantiate the MLP class
mlp = MLP(hidden_dim=100, num_classes=10)

# The compile step specifies the training configuration.
mlp.compile(optimizer=tf.train.RMSPropOptimizer(.001),
            loss=keras.losses.sparse_categorical_crossentropy,
            metrics=['accuracy'])

In [22]:
# tf.data.Dataset instance
tr_data = np.random.random((1000, 32)).astype(np.float32)
tr_label = np.random.randint(low=0, high=10, size = 1000)
tr_dataset = tf.data.Dataset.from_tensor_slices((tr_data, tr_label))
tr_dataset = tr_dataset.batch(batch_size=32)
tr_dataset = tr_dataset.repeat()

val_data = np.random.random((100, 32)).astype(np.float32)
val_label = np.random.randint(low=0, high=10, size = 100)
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
val_dataset = val_dataset.batch(batch_size=100).repeat()

tst_data = np.random.random((100, 32)).astype(np.float32)
tst_label = np.random.randint(low=0, high=10, size = 100)
tst_dataset = tf.data.Dataset.from_tensor_slices((tst_data, tst_label))
tst_dataset = tst_dataset.batch(batch_size=100)

print(tr_dataset.output_types)

(tf.float32, tf.int64)


In [23]:
# Trains for 5 epochs
mlp.fit(tr_dataset, epochs=5, steps_per_epoch=1000//32, validation_data = val_dataset, validation_steps=1)

Epoch 1/5
31/31 [==============================] - 0s 3ms/step - loss: 2.3903 - acc: 0.0000e+00 - val_loss: 2.5355 - val_acc: 0.0000e+00
Epoch 2/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3593 - acc: 0.0020 - val_loss: 2.4501 - val_acc: 0.0000e+00
Epoch 3/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3239 - acc: 0.0383 - val_loss: 2.3684 - val_acc: 0.0000e+00
Epoch 4/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3096 - acc: 0.1401 - val_loss: 2.3465 - val_acc: 0.0000e+00
Epoch 5/5
31/31 [==============================] - 0s 1ms/step - loss: 2.2976 - acc: 0.1835 - val_loss: 2.3411 - val_acc: 0.0000e+00


### Custom layers
Reading https://www.tensorflow.org/guide/keras?hl=ko#custom_layers

## Callbacks
A callback is an object passed to a model to customize and extend its behavior during training. You can write your own custom callback, or use the built-in `tf.keras.callbacks` that include:

* `tf.keras.callbacks.ModelCheckpoint`: Save checkpoints of your model at regular intervals.
* `tf.keras.callbacks.LearningRateScheduler`: Dynamically change the learning rate.
* `tf.keras.callbacks.EarlyStopping`: Interrupt training when validation performance has stopped improving.
* `tf.keras.callbacks.TensorBoard`: Monitor the model's behavior using TensorBoard.  

To use a `tf.keras.callbacks.Callback`, pass it to the model's `fit` method:

In [24]:
# Clear
keras.backend.clear_session()
tf.reset_default_graph()

print(tf.get_default_graph().get_operations())
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[]
[]


In [25]:
# tf.data.Dataset instance
tr_data = np.random.random((1000, 32)).astype(np.float32)
tr_label = np.random.randint(low=0, high=10, size = 1000)
tr_dataset = tf.data.Dataset.from_tensor_slices((tr_data, tr_label))
tr_dataset = tr_dataset.batch(batch_size=32)
tr_dataset = tr_dataset.repeat()

val_data = np.random.random((100, 32)).astype(np.float32)
val_label = np.random.randint(low=0, high=10, size = 100)
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
val_dataset = val_dataset.batch(batch_size=100).repeat()

tst_data = np.random.random((100, 32)).astype(np.float32)
tst_label = np.random.randint(low=0, high=10, size = 100)
tst_dataset = tf.data.Dataset.from_tensor_slices((tst_data, tst_label))
tst_dataset = tst_dataset.batch(batch_size=100)

print(tr_dataset.output_types)

(tf.float32, tf.int64)


In [26]:
# Creating "callback" object
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./logs')
]

# Training
model = keras.Sequential()
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=10, activation='softmax'))
model.compile(optimizer=tf.train.GradientDescentOptimizer(.01), 
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'], callbacks = callbacks)

model.fit(tr_dataset, epochs = 5, steps_per_epoch = 1000 // 32,
          validation_data = val_dataset, validation_steps = 1)

Epoch 1/5
31/31 [==============================] - 0s 3ms/step - loss: 2.3471 - acc: 0.0020 - val_loss: 2.3002 - val_acc: 0.0000e+00
Epoch 2/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3285 - acc: 0.0040 - val_loss: 2.2775 - val_acc: 0.0000e+00
Epoch 3/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3192 - acc: 0.0040 - val_loss: 2.2651 - val_acc: 0.0000e+00
Epoch 4/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3137 - acc: 0.0040 - val_loss: 2.2580 - val_acc: 0.0000e+00
Epoch 5/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3101 - acc: 0.0060 - val_loss: 2.2533 - val_acc: 0.0000e+00


## Save and restore
To be modified

### Weights only
Save and load the weights of a model using `tf.keras.Model.save_weights`:

In [27]:
# Clear
keras.backend.clear_session()
tf.reset_default_graph()

print(tf.get_default_graph().get_operations())
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[]
[]


In [28]:
# Subclassing tf.keras.Model
class MLP(keras.Model):
    def __init__(self, hidden_dim, num_classes):
        super(MLP, self).__init__()
        # Define your layers here.
        self.hidden_layer = keras.layers.Dense(units = hidden_dim, activation='relu')
        self.output_layer = keras.layers.Dense(units = num_classes, activation='softmax')
        
    def call(self, inputs):
        hidden = self.hidden_layer(inputs)
        score = self.output_layer(hidden)
        return score
    
# Instantiate the MLP class
mlp = MLP(hidden_dim=100, num_classes=10)

# The compile step specifies the training configuration.
mlp.compile(optimizer=tf.train.RMSPropOptimizer(.001),
            loss=keras.losses.sparse_categorical_crossentropy,
            metrics=['accuracy'])

In [29]:
# tf.data.Dataset instance
tr_data = np.random.random((1000, 32)).astype(np.float32)
tr_label = np.random.randint(low=0, high=10, size = 1000)
tr_dataset = tf.data.Dataset.from_tensor_slices((tr_data, tr_label))
tr_dataset = tr_dataset.batch(batch_size=32)
tr_dataset = tr_dataset.repeat()

val_data = np.random.random((100, 32)).astype(np.float32)
val_label = np.random.randint(low=0, high=10, size = 100)
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
val_dataset = val_dataset.batch(batch_size=100).repeat()

tst_data = np.ones((100,32), dtype=np.float32)
tst_label = np.ones((100,), dtype=np.int64)
tst_dataset = tf.data.Dataset.from_tensor_slices((tst_data, tst_label))
tst_dataset = tst_dataset.batch(batch_size=100)

print(tr_dataset.output_types)
print(tst_dataset.output_types)

(tf.float32, tf.int64)
(tf.float32, tf.int64)


In [30]:
# Trains for 5 epochs
mlp.fit(tr_dataset, epochs=5, steps_per_epoch=1000//32,
        validation_data=val_dataset, validation_steps=1)
print(mlp.evaluate(tst_dataset, steps=1))
# Save weights to a HDF5 file
mlp.save_weights('./weights/mlp')

Epoch 1/5
31/31 [==============================] - 0s 4ms/step - loss: 2.3604 - acc: 0.0010 - val_loss: 2.3443 - val_acc: 0.0000e+00
Epoch 2/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3436 - acc: 0.0091 - val_loss: 2.3313 - val_acc: 0.0000e+00
Epoch 3/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3231 - acc: 0.0262 - val_loss: 2.3315 - val_acc: 0.0000e+00
Epoch 4/5
31/31 [==============================] - 0s 1ms/step - loss: 2.3125 - acc: 0.0111 - val_loss: 2.3399 - val_acc: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 870us/step
[2.311108112335205, 0.0]


In [31]:
# Clear
keras.backend.clear_session()
tf.reset_default_graph()

print(tf.get_default_graph().get_operations())
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[]
[]


In [32]:
tst_data = np.ones((100,32), dtype=np.float32)
tst_label = np.ones((100,), dtype=np.int64)
tst_dataset = tf.data.Dataset.from_tensor_slices((tst_data, tst_label))
tst_dataset = tst_dataset.batch(batch_size=100)

In [33]:
# Instantiate the MLP class
test_model = MLP(hidden_dim=100, num_classes=10)
# The compile step specifies the training configuration.
test_model.compile(optimizer=tf.train.RMSPropOptimizer(.001),
            loss=keras.losses.sparse_categorical_crossentropy,
            metrics=['accuracy'])

In [34]:
test_model.load_weights('./weights/mlp')

In [35]:
test_model.evaluate(tst_dataset, steps = 1)

1/1 [==============================] - 0s 17ms/step


[2.1389548778533936, 0.0]

### Configuration only
Reading https://www.tensorflow.org/guide/keras?hl=ko#configuration_only

### Entire model
Reading https://www.tensorflow.org/guide/keras?hl=ko#entire_model

## Eager execution
Reading https://www.tensorflow.org/guide/keras?hl=ko#eager_execution

## Distribution
Reading https://www.tensorflow.org/guide/keras?hl=ko#distribution